In [18]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
import os
from PIL import Image as Img
from sklearn import metrics
import cv2
import tqdm
np.random.seed(7)

In [176]:
def get_preds_trues_image_level(all_data, classs):
    all_data_split = []
    for el in all_data:
        if classs not in el:
            continue
        if el[0] == "-":

            el = el[1:].split("-")
            el[0] = "-" + el[0]

        else:
            el = el.split("-")

        all_data_split.append(el)


    all_data_info = [(float(el[0]), el[-2] != "good") for el in all_data_split]
    true = np.array([el[1] for el in all_data_info])
    preds = np.array([el[0] for el in all_data_info])
    return preds, true


def get_auc(preds, true):
    fpr, tpr, thresholds = metrics.roc_curve(true, preds)
    # print(fpr, tpr, thresholds)

    # precision, recall, thresholds_pr = metrics.precision_recall_curve(true, preds)
    aucroc = metrics.auc(fpr, tpr)
    return aucroc

In [177]:
FOLDER = r"diffusion-classification/reconstruction/test_data/cross-entropy10-5-threshold-5-add"
all_data = os.listdir(FOLDER)
scores = {}
classes = ["tile", "grid", "leather", "wood", "carpet", "bottle", "cable", "capsule", "hazelnut", "metal_nut", "pill", "screw", "toothbrush", "transistor", "zipper"]
for classs in classes:
    aucroc = get_auc(*get_preds_trues_image_level(all_data, classs))
    scores[classs] = aucroc
scores

{'tile': 0.6722582972582972,
 'grid': 0.6094402673350042,
 'leather': 0.5163043478260869,
 'wood': 0.7333333333333334,
 'carpet': 0.6001203852327448,
 'bottle': 0.5384920634920635,
 'cable': 0.5073088455772115,
 'capsule': 0.4357798165137615,
 'hazelnut': 0.5071428571428571,
 'metal_nut': 0.5332355816226784,
 'pill': 0.5005455537370431,
 'screw': 0.5252100840336135,
 'toothbrush': 0.5833333333333334,
 'transistor': 0.55,
 'zipper': 0.5126050420168067}

In [21]:
np.mean(list(scores.values()))

0.9825

In [120]:
MASK_FOLDER = r"diffusion-segmentation\test_data_masks"
PREDS_FOLDER = r"diffusion-l1-unconditional\capsule\anomaly_maps\without-reconstructed-30-1-threshold-0-new"

In [121]:
all_data = os.listdir(PREDS_FOLDER)

In [108]:
import warnings
warnings.filterwarnings("ignore")

def get_true_preds_pixel_level(all_data, classs):
    preds = []
    true = []
    preds_paths = all_data
    masks_paths = os.listdir(MASK_FOLDER)
    set_masks_path = set(masks_paths)
    for pred_path in tqdm.tqdm(preds_paths):
        if classs not in pred_path:
            continue

        striped_path = "-".join(pred_path.strip("-").split("-")[1:])
        pred_mask = (cv2.imread(os.path.join(PREDS_FOLDER, pred_path), cv2.IMREAD_GRAYSCALE) / 255)
        if striped_path in set_masks_path:
            gt_mask = (cv2.resize(cv2.imread(os.path.join(MASK_FOLDER, striped_path), cv2.IMREAD_GRAYSCALE), (256, 256)) / 255).astype(int)
        else:
            gt_mask = np.zeros(pred_mask.shape).astype(int)



        preds.extend(pred_mask.reshape(-1))
        true.extend(gt_mask.reshape(-1))

    return preds, true

In [123]:
all_data = os.listdir(PREDS_FOLDER)
scores = {}
classes = ["tile", "grid", "leather", "wood", "carpet", "bottle", "cable", "capsule", "hazelnut", "metal_nut", "pill", "screw", "toothbrush", "transistor", "zipper"]
for classs in ["capsule"]:
    aucroc = get_auc(*get_true_preds_pixel_level(all_data, classs))
    scores[classs] = aucroc
scores

100%|██████████| 132/132 [00:07<00:00, 16.85it/s]


{'capsule': 0.5948917875808059}

In [86]:
np.mean(list(scores.values()))

0.8276390361047128